# Задача
В мобильной игре около 10% игроков совершает платежи в первый месяц с момента установки игры (база расчета - 500 игроков). Геймдизайнеры разработали обновление которое по их экспертной оценке должно увеличить процент плательщиков с 10% до 11%. Цель эксперимента - подтвердить или опровергнуть их гипотезу.

- Опишите оптимальный по вашему мнению дизайн эксперимента
- Рассчитайте длительность эксперимента при условии что каждый день в игру приходит около 100 новых игроков.
- Самостоятельно сгенерируйте датасет с около 10% плательщиков (контроль) и рассчитайте доверительный интервал.
- Сгенерируйте несколько вариантов экспериментальной группы (хуже, лучше, без эффекта), рассчитайте силу эффекта, ДИ и ваши выводы для каждого варианта.
- Решите задачу 2 разными подходами: frequentist / bayesian
Необходимо все ответы сопроводить комментариями почему вы выбрали ту или иную методику расчета, алгоритм и т.д.

# Решение

Для начала нам нужны данные, с которыми будем работать, поэтому сделаем несколько выборок клиентов, которые будем сравнивать до/после, а также с изменениями и без.

In [1]:
import numpy as np
from scipy import stats as st
from typing import List

In [2]:
np.random.seed(42)

In [3]:
# группа игроков до изменений
group_before_change = np.array([0] * 450 + [1] * 50)

# группы после
positive_group_after_change = np.array([0] * 445 + [1] * 55)
negative_group_after_change = np.array([0] * 460 + [1] * 40)
unchanged_group_after_change = np.array([0] * 450 + [1] * 50)

In [4]:
# перемешаем, чтобы выглядело более похоже на реальные данные
np.random.shuffle(group_before_change)
np.random.shuffle(positive_group_after_change)
np.random.shuffle(negative_group_after_change)
np.random.shuffle(unchanged_group_after_change)

Мы создали синтетические массивы:
- `group_before_change` — 500 пришедших игроков до изменения
- `positive_group_after_change` — 500 пришедших игроков после изменения (кол-во платящих выросло на прогнозируемые 11%)
- `negative_group_after_change` — 500 пришедших игроков после изменения (кол-во платящих наоборот упало до 8%)
- `unchanged_group_after_change` — 500 пришедших игроков после изменения (кол-во платящих совсем не изменилось)

Что еще стоит уточнить? 
- Будем считать, что до и после изменений исследуются разные игроки, соответственно, это уже независимые группы людей (новые игроки со временем становятся неновыми и уже не оцениваются повторно)
- На таких маленьких данных в целом не нужны какие-либо статистические проверки гипотез, но предположим, потом нужно будет масштабировать решение на бесконечное количество игроков.
- Будем считать, что в период проведения А/Б-тестирования не происходило никаких других нововведений, акций, PR-скандалов, попадания в топы платформодержателей или падений астероидов.
- уровень статистической значимости возьмем за 0.05

Геймдизайнеры утверждают, что нововведение посысит количество плательщиков среди игроков до 11%, но вряд ли признаются, как это посчитали. Но в качестве нулевой гипотезы мы предположим, что никаких изменений на самом деле не случилось. А альтернативная будет заключаться в том, что кол-во плательщиков изменилось. Можно попробовать рассчитать t-критерий для независимых выборок.

In [5]:
class ABTesting():
    
    df_before: np.ndarray
    df_after: np.ndarray
    alpha: float
    samples_before: List[int]
    samples_after: List[int]
    
    def __init__(self, df_before: np.ndarray, df_after: np.ndarray, alpha: float) -> None:
        self.df_before = df_before
        self.df_after = df_after
        self.alpha = alpha
        self.samples_before = []
        self.samples_after = []
        
    def ttest(self) -> None:
        # проверяем наличие входных данных
        self.__validator()
        
        # проводим бутстрапирование кол-ва заплативших игроков
        self.__bootstrap()
        
        # проводим t-тест на двух полученных выборках
        results = st.ttest_ind(self.samples_before, self.samples_after)
        if results.pvalue < self.alpha:
            print('Отвергаем нулевую гипотезу')
        else:
            print('Не получилось отвергнуть нулевую гипотезу')

    def __validator(self) -> None:
        if not hasattr(self, 'df_before'):
            raise Exception('Не переданы данные до изменения')
        if not hasattr(self, 'df_after'):
            raise Exception('Не переданы данные после изменения')
        if not hasattr(self, 'alpha'):
            self.alpha = 0.05
    
    def __bootstrap(self, sample_size: int = 50) -> None:
        np.random.seed(42)
        for _ in range(500):
            # делаем случайные выборки по 50 игроков с возвратом
            sample_1 = np.random.choice(self.df_before, size=sample_size, replace=True)
            sample_2 = np.random.choice(self.df_after, size=sample_size, replace=True)
            
            # добавляем кол-во заплативших игроков в новые списки
            self.samples_before.append(sum(sample_1))
            self.samples_after.append(sum(sample_2))
            

In [6]:
test_positive = ABTesting(group_before_change, positive_group_after_change, 0.05)
test_negative = ABTesting(group_before_change, negative_group_after_change, 0.05)
test_unchanged = ABTesting(group_before_change, unchanged_group_after_change, 0.05)

In [7]:
# эксперимент, где плательщиков стало больше
test_positive.ttest()

Отвергаем нулевую гипотезу


In [8]:
# эксперимент, где плательщиков стало меньше
test_negative.ttest()

Отвергаем нулевую гипотезу


In [9]:
# эксперимент, где кол-во плательщиков не изменилось
test_unchanged.ttest()

Не получилось отвергнуть нулевую гипотезу


В группах с изменениями мы отвергаем гипотезу о том, что выборки равны, что удовлетворяет тому, что мы знаем за пределами эксперимента. А там, где не было изменений — отвергнуть нулевую гипотезу не вышло. Однако, нам это не говорит о том, в какую сторону произошли изменения. 

Что касается длительности эксперимента, то первый попавшийся онлайн-калькулятор мне насчитал 28257 человек в экспериментальной и контрольных группах. Попробуем оценить то же вручную, при условии, что каждый день мы получаем по 100 новых игроков. Воспользуемся формулой для расчета кол-ва выборки с z-тестами.

In [10]:
alpha = 0.05 # уровень стат. значимости
beta = 0.2 # наша мощность
p_0 = 0.10  # начальная конверсия
p_1 = 0.11  # целевая конверсия
effect = p_1 - p_0  # минимально допустимое изменение

z_over_alpha = st.norm.ppf(1 - alpha / 2) # критическое z для 0.05
z_over_beta = st.norm.ppf(1 - beta) # критическое z для 80% мощности

# Расчет размера выборки
n = (2 * (z_over_alpha + z_over_beta) ** 2 * p_0 * (1 - p_0)) / effect ** 2

# Расчет длительности эксперимента
daily_new_players = 100
duration_days = n / daily_new_players

print(f"Размер выборки (для каждой группы): {round(n)}")
print(f"Длительность эксперимента (в днях): {round(duration_days)}")

Размер выборки (для каждой группы): 14128
Длительность эксперимента (в днях): 141


Получился тот же результат, согласно которому потребуется около 141 дня, чтобы максимально точно оценить изменение на 1%. Представим, что мы собрали 14128 наблюдений, в которых теперь действительно 11% плательщиков.

In [11]:
np.random.seed(42)

# группа с позитивным измерением
big_positive_group_after_change = np.array([0] * 12573 + [1] * 1554)
np.random.shuffle(big_positive_group_after_change)

Проведем односторонний тест со следующими гипотезами:
- Нулевая: кол-во платящих игроков осталось прежним или даже меньше 10%
- Альтернативная: кол-во платящих игроков действительно стало больше 10%

Сделаем 500 выборок по 100 игроков, где в среднем мы хотим проверить, что платящих игроков 10 или менее, т.е. 10%.

In [12]:
def check_equal_mean(sample: np.ndarray, value: int) -> None:
    samples_mean = []
    for _ in range(5000):
        one_sample = np.random.choice(big_positive_group_after_change, size=100, replace=True)
        samples_mean.append(sum(one_sample))

    results = st.ttest_1samp(samples_mean, value)
    
    if (results.pvalue / 2 < alpha) and (np.array(samples_mean).mean() > value):
        print("Отвергаем нулевую гипотезу")
    else:
        print("Не отвергаем нулевую гипотезу")

In [13]:
check_equal_mean(big_positive_group_after_change, 10)

Отвергаем нулевую гипотезу


Можно сказать, что изменения таким образом действительно можно зафиксировать, хотя все равно нет гарантий, что это будет работать всегда именно так, можно лишь статистически прикинуть, отвергать гипотезу или нет, но не подтвердить её. Это будет лучше делать на фактических данных всей выборки.